In [1]:
import csv
import pandas as pd
import glob
import os
from datetime import datetime
from xlsxwriter.workbook import Workbook
import sys
import xlsxwriter
from tkinter import filedialog
from tkinter import *
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import GridspecLayout
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Code zeigen / verstecken'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)
directory = ""

def change_directory(b):
    root = Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory()
    print("Changed directory to: " + folder_selected)
    global directory
    directory=folder_selected
def start_processing(b):
    combine_csv(directory, text_field.value)
    
button_start = widgets.Button(description = 'Start', disabled = False)
button_direc = widgets.Button(description =  'Choose Folder', disabled = False)
button_start.on_click(start_processing)
button_direc.on_click(change_directory)
text_field = widgets.Text(value = '', placeholder = 'combined_file', description = 'Name Export')
grid = GridspecLayout(2,2)
grid[0,0] = text_field
grid[1,1] = button_start
grid[0,1] = button_direc 
display(grid)
def combine_csv(dir_, name):
    
    mydir=dir_
    files=glob.glob(os.path.join(mydir, "*.csv"))
    dataframes = []

    for csvfile in files:
        df=pd.read_csv(csvfile)
        print(csvfile)
        dataframes.append(df)

    result=pd.concat(dataframes, ignore_index=True)
    result.to_csv(mydir+'\\all.csv', index=False, encoding="utf-8-sig")

    f=open(mydir+"\\all.csv", encoding="utf8")
    workbook=xlsxwriter.Workbook(mydir+f"\\{name}.xlsx")
    worksheet=workbook.add_worksheet()
    rem_words=["Herr ","Dipl Designer","dent","rerpol", "MBA", "Dr. med.", "Dr.", "Dipl. Ing. ", "Prof. Dr. ", "Prof", ".", "DiplIng", "Dipl- Ing.", "Dipl-Ing"]
    i=1
    try:
        reader=csv.reader(f)
        worksheet.write(0,0, "Name")
        worksheet.write(0,1, "Vorname")
        worksheet.write(0,2,"Umsatz")
        worksheet.write(0,3,"Ziffern")
        worksheet.write(0,4,"Katalog")
        worksheet.write(0,5,"Versicherungstyp")
        worksheet.write(0,6,"Alter")
        worksheet.write(0,7,"Leistung")
        worksheet.write(0,8,"PLZ")
        worksheet.write(0,9,"Praxisname")
        worksheet.write(0,10,"Arzt")
        worksheet.write(0,11,"Patienten Nr. ")
        worksheet.write(0,12,"Erstes Leistungsdatum")
        for row in reader:
            liste=[]
            if i > 1:
                nachname=row[0].split(", ")[0]
                vorname=row[0].split(", ")[1]
                for item in rem_words: 
                    nachname=nachname.replace(item,"")
                nachname=nachname.strip()
                liste.append(nachname)
                liste.append(vorname)
                for n in range(10):
                    liste.append(row[n+1].strip())
                liste.append(row[11].split(",")[0])
                j=0
                for eintrag in liste:
                    worksheet.write(i,j,eintrag)
                    j=j+1
            i=i+1
    finally:
        f.close()

    print("finished writing xlsx file!")
    workbook.close()
hide_toggle()

GridspecLayout(children=(Text(value='', description='Name Export', layout=Layout(grid_area='widget001'), place…

Changed directory to: C:/Users/kai.follmann/CSVCombiner
C:/Users/kai.follmann/CSVCombiner\Patientenbuch_20210827_1421 (1).csv
C:/Users/kai.follmann/CSVCombiner\Patientenbuch_20210827_1421.csv
C:/Users/kai.follmann/CSVCombiner\Patientenbuch_20210827_1422.csv
finished writing xlsx file!
